# 실습1: 이미지 데이터 확인

In [ ]:
import  pandas as pd
import numpy as np

import PIL
import matplotlib.image as img
import matplotlib.pyplot as plt

from elice_utils import EliceUtils
elice_utils = EliceUtils()

# 이미지 목록을 불러오는 함수입니다.

def load_data(path):
    return pd.read_csv(path)

'''
1. PIL.Image를 이용하여 이름(경로+이름)을 바탕으로 이미지를 불러오고,
   이를 리스트 'images'에 추가하는 함수를 완성합니다.
   main 함수에서 'path'와 'names' 변수가 무엇에 해당하는지 확인해보세요.
'''


def load_images(path, names):
    
    images=[]
    
    for name in names:
        images.append(PIL.Image.open(path+name))
    
    return images


'''
2. 이미지의 사이즈를 main 함수에 있는 'IMG_SIZE'로 
   조정하고, 이를 Numpy 배열로 변환하는 함수를 완성합니다.
'''


def images2numpy(images, size):
    
    output=[]
    
    for img in images:
        img = img.resize(size)
        np_img = np.array(img)
        output.append(np_img)     
    
    return output



# 이미지에 대한 정보를 나타내주는 함수입니다.

def sampleVisualize(np_images):

    fileName = "./data/images/1000092795.jpg"

    ndarray = img.imread(fileName)
    
    plt.imshow(ndarray)
    plt.show()    
    plt.savefig("plot.png")
    
    print("\n1-1. 'fileName' 이미지(원본): ")
    elice_utils.send_image("plot.png")
    
    print('\n1-2. Numpy array로 변환된 원본 이미지:', ndarray)
    print('\n1-3. Numpy array로 변환된 원본 이미지의 크기:', np.array(ndarray).shape)
    
    plt.imshow(np_images[0])
    plt.show()
    plt.savefig("plot_re.png")
    
    print("\n2-1. 'fileName' 이미지(resize 후): ")
    elice_utils.send_image("plot_re.png")
    
    print('\n2-2. Numpy array로 변환된 resize 후 이미지:', np_images[0])
    print('\n2-3. Numpy array로 변환된 resize 후 이미지 크기:', np.array(np_images[0]).shape)    
    
    print('\n3. Numpy array로 변환된 resize 후 이미지 10장의 크기:', np.array(np_images).shape)


'''
3. main 함수를 완성하세요.


   Step01. 이미지를 불러오는 함수를 이용해 'images'를 정의합니다.
   
   Step02. 이미지를 Numpy 배열로 바꾸는 함수를 이용해 'np_images'를 정의합니다.
'''



def main():
    
    CSV_PATH = "./data/data.csv"
    IMG_PATH = "./data/images/"
    IMG_SIZE = (300,300)
    MAX_LEN = 30
    BATCH_SIZE = 2
    
    name_caption = load_data(CSV_PATH)
    names = name_caption['file_name']
    
    images = load_images(IMG_PATH,names)
    np_images = images2numpy(images, IMG_SIZE)
    
    sampleVisualize(np_images)
    
    return images, np_images
    
if __name__=='__main__':
    main()

# 실습2: MLP로 이미지 학습

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

'''
1. MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.

   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.


           원본은 0~255 사이의 값입니다.
           
   Step02. 0~9 사이 값인 레이블을 클래스화 하기 위해 원-핫 인코딩을 진행합니다.
'''





def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
    train_images = train_images.astype(np.float32) / 255.
    test_images = test_images.astype(np.float32) / 255.
    
    train_labels = to_categorical(train_labels, 10)
    test_labels = to_categorical(test_labels, 10)    
    
    return train_images, test_images, train_labels, test_labels



'''
2. 다층 퍼셉트론(MLP) 모델을 생성합니다.
'''


def MLP():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    
    return model
    
'''
3. 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.





   Step01. MLP 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
           단, 'epochs'이 클수록, 'batch_size'는 작을수록 학습 속도가 느립니다.
   
   Step05. 모델을 테스트하고 손실(loss)값과 Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 모델의 성능을 확인해보세요.
'''





def main():
    
    train_images, test_images, train_labels, test_labels = preprocess()
    
    model = MLP()
    
    model.summary()
    
    model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])
    
    history = model.fit(train_images, train_labels, epochs = 3, batch_size = 500, validation_data = (test_images, test_labels), verbose = 0)
    
    loss, test_acc = model.evaluate(test_images, test_labels)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images))
    
    Visulaize([('MLP', history)], 'loss')
    
    return history
    
if __name__ == "__main__":
    main()

# 실습3: CNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from visual import *
from plotter import *

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# 동일한 실행 결과 확인을 위한 코드입니다.
np.random.seed(123)
tf.random.set_seed(123)

'''

1. MNIST 데이테 셋을 전처리하는 'preprocess' 함수를 완성합니다.
   
   Step01과 Step03은 이전 실습과 동일한 코드를 사용할 수 있습니다.


   Step01. MNIST 데이터 이미지를 0~1 사이 값으로 정규화해줍니다.

           원본은 0~255 사이의 값입니다.
           
   Step02. MNIST 데이터의 채널 차원을 추가해줍니다.
           
   Step03. 0~9 사이 값인 레이블을 클래스화 하기 위해 원-핫 인코딩을 진행합니다.
'''



def preprocess():
    
    # MNIST 데이터 세트를 불러옵니다.
    mnist = tf.keras.datasets.mnist
    
    # MNIST 데이터 세트를 Train set과 Test set으로 나누어 줍니다.
    (train_images, train_labels), (test_images, test_labels) = mnist.load_data()
    
    train_images, train_labels = train_images[:5000], train_labels[:5000]
    test_images, test_labels = test_images[:1000], test_labels[:1000]
    
    train_images = train_images / 255.
    test_images = test_images / 255.
    
    train_images = tf.expand_dims(train_images, -1)
    test_images = tf.expand_dims(test_images, -1)
        
    train_labels = tf.one_hot(train_labels, depth = 10)
    test_labels = tf.one_hot(test_labels, depth = 10)
    
    return train_images, test_images, train_labels, test_labels


'''
2. CNN 모델을 생성합니다.
'''

def CNN():
    
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'SAME', input_shape = (28,28,1)))
    model.add(tf.keras.layers.MaxPool2D(padding = 'SAME'))
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding = 'SAME'))
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'SAME'))
    model.add(tf.keras.layers.MaxPool2D(padding = 'SAME'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation = 'relu'))
    model.add(tf.keras.layers.Dense(32, activation = 'relu'))
    model.add(tf.keras.layers.Dense(10, activation = 'softmax'))
    
    return model
    
'''
3. 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.



   Step01. CNN 함수를 통해 모델을 불러옵니다.
   
   Step02. 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 모델을 학습시킵니다. 검증용 데이터도 설정하세요.
           'epochs'와 'batch_size'도 자유롭게 설정하세요.
           단, 'epochs'이 클수록, 'batch_size'는 작을수록 학습 속도가 느립니다.
   
   Step05. 모델을 테스트하고 손실(loss)값과 Test Accuracy 값 및 예측 클래스, 
           손실 함수값 그래프를 출력합니다. 모델의 성능을 확인해보고,
           목표값을 달성해보세요.
'''



def main():
    
    train_images, test_images, train_labels, test_labels = preprocess()
    
    model = CNN()
    
    model.summary()
    
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam',
                  metrics = ['accuracy'])
    
    history = model.fit(train_images, train_labels, epochs = 20, batch_size = 512, verbose=0, validation_data = (test_images, test_labels))
    
    loss, test_acc = model.evaluate(test_images, test_labels, verbose = 0)
    
    print('\nTest Loss : {:.4f} | Test Accuracy : {}'.format(loss, test_acc))
    print('예측한 Test Data 클래스 : ',model.predict_classes(test_images)[:10])
    
    Visulaize([('CNN', history)], 'loss')
    
    Plotter(test_images, model)
    
    return history
    
if __name__ == "__main__":
    main()

# 실습4: 토큰화와 형태소 분석

In [ ]:
import numpy as np
from konlpy.tag import Twitter 

def load_data(path):
    with open(path, 'r') as f:
        data = f.read()
    return data

'''
1. '문서'를 '문단'의 리스트로 변환하는 함수 doc2para를 완성합니다.

   Step01. 문장의 마침표(.) 뒤에 있는 개행 표시(\n)를 기준으로 

           문서 내 글들을 리스트 요소 즉, 문단으로 나눕니다.
           
   Step02. 나누어진 글들 중 마지막 글자가 "."인 경우만 
           문단이 나누어진 것으로 보고 그 외의 문장들은 서로 
           병합하여 줍니다.
'''



def doc2para(writing):
    
    paragraphs = []
    
    splited = writing.split('\n')
    splited = list(filter(lambda x: len(x)>0, splited))
    para = ""
    
    for sentence in splited:
        if sentence[-1] != '.':
            para += sentence
        else:
            # 아래 코드를 추가하여 문단 내 마지막 문장을 읽어올 수 있습니다.
            para += sentence
            paragraphs.append(para)
            para = ""
    
    return paragraphs


'''
2. '문단'을 '문장'의 리스트로 변환하는 함수 para2sen을 완성합니다.

   Step01. 문단을 "."으로 나누어 리스트로 만들고, 

           변수 sentences에 저장합니다.
           
   Step02. sentences 내 문장들에 대해서 "?"로 재분할 한 후, 
           ndarray.flatten()을 활용하여 재분할된 문장이 합쳐질 
           수 있도록 리스트로 만들어 줍니다. 
           ("!"에 대해서도 마찬가지로 적용합니다.)
'''



def para2sen(paragraph):
    
    sentences = []
    
    # Step01.
    sentences = paragraph.split('.')
    
    # Step02. 
    sentences = [sentence.split('?') for sentence in sentences]
    sentences = np.array(sentences).flatten()
    sentences = [sentence.split('!') for sentence in sentences]
    sentences = np.array(sentences).flatten()
    sentences = [ sentence.replace('"','') for sentence in sentences]
    
    return sentences


# 띄어쓰기로 문장을 구분하는 함수


def sen2words_byspace(sentence):
    
    words = []
    words = sentence.strip().split(" ")
    
    return words


'''
3. 'Twitter()'로 선언된 Tokenizer인 'analyzer'를 이용해 형태소에 따라 
    분할된 문장의 리스트를 변수 'morphs'에 저장하는 sen2morph
    함수를 완성합니다. Twitter.morphs 메소드를 사용하세요.
'''


def sen2morph(sentence):
    
    morphs = []
    
    analyzer = Twitter()
    morphs = analyzer.morphs(sentence)
    
    return morphs


'''
4. 3번과 같이 'Twitter()'로 선언된 Tokenizer인 'analyzer'를 이용해
   형태소와 그에 따른 품사를 분할하는 analyzing_morphs 함수를 완성합니다.
   Twitter.pos 메소드를 사용하세요.
'''


def analyzing_morphs(sentence):
    
    analyzer = Twitter()
    
    return analyzer.pos(sentence)
    
# 위에서 정의한 함수들을 바탕으로 문서를 토큰화를 진행합니다.



def main():
    
    DATA_PATH = "./data/blood_rain.txt"
    
    blood_rain = load_data(DATA_PATH)
    paragraphs = doc2para(blood_rain)
    sentences = para2sen(paragraphs[4])
    words_byspace = sen2words_byspace(sentences[3])
    words_bymorphs = sen2morph(sentences[3])
    morphs_analyzed = analyzing_morphs(sentences[3])
    
    # 출력을 통해 토큰화가 잘 되었는지 확인합니다.
    
    print("문장으로 구분된 5번째 문단: ", sentences)
    print("\n띄어쓰기로 구분된 문장 (5번째 문단의 4번째 문장): ", words_byspace)
    print("\n형태소 별로 구분된 문장 (5번째 문단의 4번째 문장): ", words_bymorphs)
    print("\n형태소와 그에 따른 품사로 분류된 문장 (5번째 문단의 4번째 문장): ", morphs_analyzed)
    
    return words_byspace, words_bymorphs, morphs_analyzed
    
if __name__=='__main__':
    main()

# 실습5: Bag of Words

In [ ]:
import numpy as np
from konlpy.tag import Twitter

# 텍스트 파일을 읽어서 저장하는 함수입니다.


def read_txt(path):
    
    file = open(path, 'r')
    output = str(file.read())
    
    return output


'''
1. BoW를 딕셔너리 형태로 출력하는 함수를 만들어봅니다.

   Step01. 기존 word_dict 에 그 값이 있으면 1을 더해주고, 없으면 1을 부여합니다.
'''

def bag_of_words(tokenized_sentences):
    word_dict = {}
    for tokenized_sentence in tokenized_sentences:
        for token in tokenized_sentence:
            try:
                word_dict[token]+=1
            except:
                word_dict[token]=1
    return word_dict

'''
2. 읽어온 텍스트 파일을 형태소 단위로 분석해서 저장하는 
   함수를 완성하세요. 이전 실습을 참고해도 좋습니다.
'''


def get_splited_doc(path):
    
    output = []
    
    text = read_txt(path)
    analyzer = Twitter()
    output = analyzer.morphs(text)
    
    return text, output



def main():
    
    PATH = "./data/text.txt"
    
    origin, splitted = get_splited_doc(PATH)
    
    print('형법 제2장 원본: ', origin)
    
    bow_criminal_law = bag_of_words([splitted])
    
    print('\n형법 제2장의 BoW: ', bow_criminal_law)
    
    return bow_criminal_law


if __name__ == "__main__":
    main()

# 실습6: 임베딩

In [ ]:
import tensorflow as tf
from tensorflow.python.keras.preprocessing.text import Tokenizer

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

'''
1. embedding 함수를 완성합니다.


   Step01. 입력된 리스트에 존재하는 요소마다 고유 인덱스를 붙입니다.
           
   Step02. 요소와 인덱스를 짝지은 딕셔너리 'word_dict'를 정의합니다.
   
   Step03. 'sentence1', 'sentence2'를 정수값으로 변환하고 이를
           각각 리스트 변수 'sen1', 'sen2'로 정의합니다.
'''



def embedding(sentence1, sentence2):
    
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(sentence1+sentence2)
    word_dict = tokenizer.word_index
    # word_dict의 value가 0부터 시작하게 바꿔줍니다.
    for k, v in word_dict.items():
        word_dict[k] = v - 1
    
    sen1 = tokenizer.texts_to_sequences(sentence1)
    sen2 = tokenizer.texts_to_sequences(sentence2)
    
    sen1 = [token[0] for token in sen1]
    sen2 = [token[0] for token in sen2]
    
    return word_dict, sen1, sen2
    
'''
2. 텐서플로우를 사용하여 원-핫 인코딩을 실행합니다.
   원-핫 벡터의 총 길이는 'word_dict' 안의 word의 총 개수입니다. 
   단어를 원-핫 인코딩 후 이것을 문장별로 (요소별) 더합니다.
'''   



def one_hot(sen1, sen2, word_dict):
    
    oh_sen1 = sum(tf.one_hot(sen1, len(word_dict)))
    oh_sen2 = sum(tf.one_hot(sen2, len(word_dict)))
    
    return oh_sen1, oh_sen2



def main():
    
    sentence1 = ['나','는','오늘','저녁','에','치킨','을','먹','을','예정','입니다']
    sentence2 = ['나','는','어제', '맥주','와', '함께', '치킨','을', '먹었', '습니다']
    
    word_dict, seq_1, seq_2 = embedding(sentence1, sentence2)
    onehot_sen1, onehot_sen2 = one_hot(seq_1, seq_2, word_dict)
        
    print('리스트 요소-인덱스 딕셔너리: ', word_dict)
    
    print('\n정수값으로 변환된 sentence1:', seq_1)
    print('\n정수값으로 변환된 sentence2:', seq_2)
    
    print('\n원-핫 인코딩된 문장1:', onehot_sen1.numpy())
    print('\n원-핫 인코딩된 문장2:', onehot_sen2.numpy())
    
    return onehot_sen1, onehot_sen2


if __name__ == '__main__':
    main()

# 실습7: word2vec

In [ ]:
from gensim.models import word2vec
import list_file

'''
1. CBOW 함수를 설명을 참고하여 완성하세요.
'''


def CBOW(sentences):
    
    model_cbow = word2vec.Word2Vec(sentences, size = 300, min_count = 1, window = 10, sg = 0)
    
    return model_cbow


'''
2. Skip_Gram 함수를 설명을 참고하여 완성하세요.
'''


def Skip_Gram(sentences):
    
    model_skipgram = word2vec.Word2Vec(sentences, size = 300, min_count = 1, window = 10, sg = 1)
    
    return model_skipgram


'''
3. 각 모델의 결괏값을 정의하세요.
'''


def main():
    
    sen1, sen2 = list_file.sen1(), list_file.sen2()
    
    sentences = [sen1, sen2]
    
    cbow = CBOW(sentences)
    skipgram = Skip_Gram(sentences)
    
    idx2word_set_cbow = cbow.wv.index2word
    idx2word_set_skipgram = skipgram.wv.index2word
    
    print('CBOW: ', idx2word_set_cbow)
    print('\nSkip-Gram: ', idx2word_set_skipgram)
    
    return idx2word_set_cbow, idx2word_set_skipgram


if __name__ == '__main__':
    main()

# 실습8: RNN

In [ ]:
import numpy as np

np.random.seed(100)

'''
1. 간단한 RNN 모델을 만들어봅시다.


   Step01. 0의 값을 갖는 (output_size,) 모양의 'state' 벡터를 만들어 봅니다.
   
   Step02. 1의 값을 갖는 (output_size, input_size) 모양의 'w' 벡터를 만들어 봅니다.
   
   Step03. 1의 값을 갖는 (output_size, output_size) 모양의 'u' 벡터를 만들어 봅니다.
   
   Step04. 임의의 값을 갖는 (output_size,) 모양의 'b' 벡터를 만들어 봅니다.
   
   Step05. bias 가 False 이면 b를 (output_size,) 모양의 영벡터로 만듭니다.
   
   Step06. Numpy를 사용해서 w와 _input을 내적하고, u 와 state를 내적한 후
           b를 더한 다음 tanh 함수를 적용합니다.
'''


def rnn(inputs, input_size, output_size, bias = False):

    input_size = len(inputs[0])
    
    state = np.zeros((output_size,))
    
    w = np.ones((output_size, input_size))
    
    u = np.ones((output_size, output_size))
    
    b = np.random.random((output_size,))
    
    if not bias:
        b = np.zeros((output_size,))
    
    outputs = []
    
    for _input in inputs:
        
        _output = np.tanh(np.dot(w, _input) + np.dot(u, state) + b)
        outputs.append(_output)
        state = _output
        
    return np.stack(outputs, axis=0) 


# 케이스에 따라 RNN 모델의 결과가 어떻게 바뀌는지 확인해보세요.


def main():
    
    print("-----------------CASE 1-----------------")
    _input1 = [[0], [0], [0], [0], [0]]
    
    # 입력이 모두 0이고 출력 벡터의 크기가 1일 때 값의 추세가 어떠한지 확인해보세요.
    case1_a = rnn(_input1, input_size=1, output_size=1)
    print('\nCASE 1_a:', case1_a)
    # Bias 가 있으면 값이 어떻게 변화하는지 확인해보세요.
    case1_b = rnn(_input1, input_size=1, output_size=1, bias = True)
    print('\nCASE 1_b:', case1_b)
    
    
    print("\n-----------------CASE 2-----------------")
    _input2 = [[1], [1], [1], [1], [1]]
    
    # 입력이 모두 1이고 출력 벡터의 크기가 1일 때 값의 추세가 어떠한지 확인해보세요.
    case2_a = rnn(_input2, input_size=1, output_size=1)
    print('\nCASE 2_a:', case2_a)
    # Bias 가 있으면 값이 어떻게 변화하는지 확인해보세요.
    case2_b = rnn(_input2, input_size=1, output_size=1, bias = True)
    print('\nCASE 2_b:', case2_b)
    
    
    print("\n-----------------CASE 3-----------------")
    _input3 = [[1], [2], [3], [4], [5]]
    
    # 입력값이 증가하고 출력 벡터의 크기가 2일 때 값의 추세가 어떠한지 확인해보세요.
    case3_a = rnn(_input3, input_size=1, output_size=2)
    print('\nCASE 3_a:', case3_a)
    # Bias 가 있으면 값이 어떻게 변화하는지 확인해보세요.
    case3_b = rnn(_input3, input_size=1, output_size=2, bias = True)
    print('\nCASE 3_b:', case3_b)
    
    return case1_a, case1_b, case2_a, case2_b, case3_a, case3_b


if __name__ == '__main__':
    main()

# 실습9: RNN keras

In [ ]:
import numpy as np
import tensorflow as tf
from keras.datasets import imdb
from keras.preprocessing import sequence

import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

np.random.seed(0)
tf.random.set_seed(0)

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# 데이터를 불러오고 전처리하는 함수입니다.


def load_data(n_of_training_ex, n_of_testing_ex, max_review_length):
    
    PATH = "./data/"
    
    X_train = np.load(PATH + "X_train.npy")[:n_of_training_ex]
    y_train = np.load(PATH + "y_train.npy")[:n_of_training_ex]
    X_test = np.load(PATH + "X_test.npy")[:n_of_testing_ex]
    y_test = np.load(PATH + "y_test.npy")[:n_of_testing_ex]
    
    X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
    X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
    
    return X_train, y_train, X_test, y_test
    
'''
1. SimpleRNN을 적용할 하나의 모델을 자유롭게 생성합니다.
'''
    
def SimpleRNN(embedding_vector_length, max_review_length):
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(1000, embedding_vector_length, input_length = max_review_length))
    model.add(tf.keras.layers.SimpleRNN(5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model


'''
2. LSTM을 적용할 하나의 모델을 자유롭게 생성합니다.
'''


def LSTM(embedding_vector_length, max_review_length):
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(1000, embedding_vector_length, input_length = max_review_length))
    model.add(tf.keras.layers.LSTM(5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model


'''
3. GRU를 적용할 하나의 모델을 자유롭게 생성합니다.
'''


def GRU(embedding_vector_length, max_review_length):
    
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Embedding(1000, embedding_vector_length, input_length = max_review_length))
    model.add(tf.keras.layers.GRU(5))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model


'''
4. 세 모델을 불러온 후 학습시키고 테스트 데이터에 대해 평가합니다.


   Step01. SimpleRNN, LSTM, GRU 함수를 이용해 세 모델을 불러옵니다.
   
   Step02. 세 모델의 손실 함수, 최적화 알고리즘, 평가 방법을 설정합니다.
   
   Step03. 세 모델의 구조를 확인하는 코드를 작성합니다.
   
   Step04. 세 모델을 각각 학습시킵니다. 검증용 데이터는 설정하지 않습니다.
           세 모델 모두 'epochs'는 3, 'batch_size'는 256으로 설정합니다.
   
   Step05. 세 모델을 테스트하고 각각의 Test Accuracy 값을 출력합니다. 
           셋 중 어느 모델의 성능이 가장 좋은지 확인해보세요.
'''



def main():
    
    max_review_length = 300
    embedding_vector_length = 32
    
    n_of_training_ex = 25000
    n_of_testing_ex = 3000
    
    X_train, y_train, X_test, y_test = load_data(n_of_training_ex, n_of_testing_ex, max_review_length)
    
    model_simple_rnn = SimpleRNN(embedding_vector_length, max_review_length)
    model_lstm = LSTM(embedding_vector_length, max_review_length)
    model_gru = GRU(embedding_vector_length, max_review_length)
    
    model_simple_rnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model_lstm.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model_gru.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    model_simple_rnn.summary()
    model_lstm.summary()
    model_gru.summary()
    
    model_simple_rnn_history = model_simple_rnn.fit(X_train, y_train, epochs = 3, batch_size = 64, verbose = 0)
    print('\n')
    model_lstm_history = model_lstm.fit(X_train, y_train, epochs = 3, batch_size = 64, verbose = 0)
    print('\n')
    model_gru_history = model_gru.fit(X_train, y_train, epochs = 3, batch_size = 64, verbose = 0)
    
    scores_simple_rnn = model_simple_rnn.evaluate(X_test, y_test, verbose = 0)
    scores_lstm = model_lstm.evaluate(X_test, y_test, verbose= 0)
    scores_gru = model_gru.evaluate(X_test, y_test, verbose = 0)
    
    print('\nTest Accuracy_simple rnn: ', scores_simple_rnn[-1])
    print('Test Accuracy_lstm: ', scores_lstm[-1])
    print('Test Accuracy_gru: ', scores_gru[-1])
    
    return model_simple_rnn_history, model_lstm_history, model_gru_history


if __name__ == '__main__':
    main()